In [6]:
from client import Client
from global_model import Global_model
from sklearn.utils import shuffle
from supported_modles import Supported_modles
import numpy as np
import utils

import random

In [7]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [8]:
selected_model = Supported_modles.logistic_regression

In [9]:
#NODEs
clients = utils.set_data()

In [10]:
f1_before = []
for client in clients:
    client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
    client.split_data()
    # client.prep_data()
    client.train_model(selected_model)
    print(f'Model accuracy: {client.accuracy}')
    print(f'Model f1 score: {client.F1}')
    f1_before.append(client.F1)
    print(f'Total size: {client.x.shape[0]}')
    print(f'Totla sum of malicious packets: {client.y.sum()}')
    print('-------------------------------------------------')

Model accuracy: 0.723939393939394
Model f1 score: 0.7694838056680162
Total size: 13400
Totla sum of malicious packets: 7439
-------------------------------------------------
Model accuracy: 0.7177272727272728
Model f1 score: 0.7657487740475292
Total size: 13400
Totla sum of malicious packets: 7399
-------------------------------------------------
Model accuracy: 0.7212121212121212
Model f1 score: 0.769307923771314
Total size: 6700
Totla sum of malicious packets: 3739
-------------------------------------------------
Model accuracy: 0.713030303030303
Model f1 score: 0.7582333418432473
Total size: 6700
Totla sum of malicious packets: 3647
-------------------------------------------------
Model accuracy: 0.7256445047489823
Model f1 score: 0.77367360644728
Total size: 14962
Totla sum of malicious packets: 8179
-------------------------------------------------


In [11]:
global_model = Global_model(selected_model)
global_model.init_global_model(client1.x.shape[1], selected_model, client1.model)

In [12]:
number_of_rounds = 10
for _ in range(number_of_rounds):
    RANDOM_NUMBER_OF_CLIENTS = random.randint(1, 1)

    print(f'Starting new round!')
    f1s = np.empty((0,RANDOM_NUMBER_OF_CLIENTS), float)
    data_weights = arr = np.empty((0,RANDOM_NUMBER_OF_CLIENTS), float)

    applicable_clients = random.sample((clients), RANDOM_NUMBER_OF_CLIENTS)

    for client in applicable_clients:
        print(f'Client name: {client.name}')
        client.test_model_f1()
        f1s = np.append(f1s, client.F1)
        data_weights = np.append(data_weights, client.x.shape[0])
    
    # print(f'Weights: {f1s/sum(f1s)}')
    # print(f'weigths data: {data_weights/sum(data_weights)}')
    
    round_weights = (f1s/sum(f1s) + data_weights/sum(data_weights))/2
    
    # print(round_weights)

    global_model.update_global_model(applicable_clients,round_weights,selected_model)

    for client in clients:
        f1_fede = global_model.f1_score(client.x_test,client.y_test)
        f1_local = client.test_model_f1()

        if f1_fede > f1_local:
            print("f1 fede was better")
            client.F1 = f1_fede
            # local was better set to local values
            if selected_model == Supported_modles.MLP_classifier:
                client.model.intercepts_ = global_model.model.intercepts_
                client.model.coefs_ = global_model.model.coefs_
            else:
                client.model.intercept_ = global_model.model.intercept_
                client.model.coef_ = global_model.model.coef_
                
    print(f'++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

Starting new round!
Client name: node3
f1 fede was better
f1 fede was better
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node4
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node4
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node4
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node1
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node1
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node4
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node3
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round

In [14]:
i = 0
for client in clients:
    print(client.F1)
    f1_before[i]
    print(f'Difference {client.F1-f1_before[i]}')
    i = i + 1

0.7714213709677419
Difference 0.0019375652997256632
0.7657487740475292
Difference 0.0
0.769307923771314
Difference 0.0
0.7621457489878543
Difference 0.003912407144606944
0.77367360644728
Difference 0.0
